## Self-attention Generative Adversarial Capsule Networks

#### Importing Modules

In [13]:
import os, sys, glob
import numpy as np
import zipfile
import skimage
from matplotlib import pyplot as plt
from PIL import Image
import tensorflow as tf
from keras import layers, models, optimizers
from keras.layers import Input
from keras.models import Sequential, Model
from keras import callbacks, backend
from keras.utils import to_categorical
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
from utils import load_image, load_faces
from discriminator import discriminator_func
from generator import generator_func
from attention import self_attention

#### Dataset Preparation

In [ ]:
!wget https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/celeba.zip

with zipfile.ZipFile("celeba.zip","r") as zip_ref:
  zip_ref.extractall("data_faces/")

root = 'data_faces/img_align_celeba'
img_list = os.listdir(root)
print("Total Number of Images : ", len(img_list))
required_size = (32, 32)

In [14]:
number_of_images = 8000
 

root = 'data_faces/img_align_celeba/'
dataset = load_faces(directory, number_of_images)
print('Loaded: ', dataset.shape)

NameError: name 'directory' is not defined

In [15]:
def generate_real_samples(dataset, n_samples):
	ix = np.random.randint(0, dataset.shape[0], n_samples)
	X = dataset[ix]
	y = np.ones((n_samples, 1))
	return X, y
 

def generate_latent_points(latent_dim, n_samples):
	x_input = np.random.randn(latent_dim * n_samples)
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input
 

def generate_fake_samples(generator, latent_dim, n_samples):
	x_input = generate_latent_points(latent_dim, n_samples)
	X = generator.predict(x_input)
	y = np.zeros((n_samples, 1))
	return X, y

#### Building Discriminator

In [16]:
input_shape = (32, 32, 3)

discriminator = discriminator_func(input_shape)

print('DISCRIMINATOR')

discriminator.summary()

discriminator.compile(loss = 'binary_crossentropy', optimizer = Adam(0.0004, 0.48), metrics = ['accuracy'])

DISCRIMINATOR
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 28, 28, 16)   1216        input_5[0][0]                    
__________________________________________________________________________________________________
self_attention_7 (self_attentio (None, 28, 28, 16)   321         conv1[0][0]                      
__________________________________________________________________________________________________
leaky_re_lu_14 (LeakyReLU)      (None, 28, 28, 16)   0           self_attention_7[0][0]           
______________________________________________________________________________

#### Buidling Generator from Random Samples

In [17]:
generator = generator_func(100)

print('GENERATOR')

generator.summary()

GENERATOR
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 16384)             1654784   
_________________________________________________________________
leaky_re_lu_19 (LeakyReLU)   (None, 16384)             0         
_________________________________________________________________
reshape_3 (Reshape)          (None, 8, 8, 256)         0         
_________________________________________________________________
gaussian_noise_3 (GaussianNo (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_transpose_7 (Conv2DTr (None, 16, 16, 16)        102416    
_________________________________________________________________
leaky_re_lu_20 (LeakyReLU)   (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_transpose_8 (Conv2DTr (None, 32, 32, 

In [18]:
z = Input(shape=(100,))

img = generator(z)

discriminator.trainable = False

valid = discriminator(img)

combined = Model(z, valid)

print('COMBINED')

combined.summary()

combined.compile(loss='binary_crossentropy', optimizer=Adam(lr = 0.0004))

def define_gan(generator, discriminator):

	discriminator.trainable = False

	model = Sequential()

	model.add(generator)

	model.add(discriminator)

	opt = Adam(lr=0.0001)
    
	model.compile(loss='binary_crossentropy', optimizer = opt)
    
	return model


COMBINED
Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 100)               0         
_________________________________________________________________
sequential_3 (Sequential)    (None, 64, 64, 3)         1780453   
_________________________________________________________________
model_3 (Model)              (None, 1)                 8602147   
Total params: 10,382,600
Trainable params: 1,780,453
Non-trainable params: 8,602,147
_________________________________________________________________
